In [ ]:
import grpc
import swoq_pb2
import swoq_pb2_grpc
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
player_id = '6616b1c5bd0a697480a68319'

In [ ]:
credentials = grpc.ssl_channel_credentials(open('localhost.pem', 'rb').read())
with grpc.secure_channel('localhost:7262', credentials) as channel:
    stub = swoq_pb2_grpc.TrainingStub(channel)
    startResponse = stub.StartGame(swoq_pb2.StartRequest(playerId=player_id))

print(f'{startResponse.result=}')

game_id = startResponse.gameId
print(f'{game_id=}')

state = np.array(startResponse.state).reshape(20,20)
plt.figure()
plt.imshow(state)

In [ ]:
credentials = grpc.ssl_channel_credentials(open('localhost.pem', 'rb').read())
with grpc.secure_channel('localhost:7262', credentials) as channel:
    stub = swoq_pb2_grpc.TrainingStub(channel)
    moveResponse = stub.Move(swoq_pb2.MoveRequest(gameId=game_id, direction=swoq_pb2.EAST))

print(f'{moveResponse.result=}')

state = np.array(moveResponse.state).reshape(20,20)
plt.figure()
plt.imshow(state)

In [ ]:
import math

In [ ]:
STATE_UNKNOWN = 0
STATE_PLAYER = 1
STATE_EMPTY = 2
STATE_WALL = 3
STATE_EXIT = 4

In [ ]:
playerPos_x = 1
playerPos_y = 1

pos_x = 5
pos_y = 4

srcX = playerPos_x + 0.5
srcY = playerPos_y + 0.5
dstX = pos_x + 0.5
dstY = pos_y + 0.5

dx = dstX - srcX
dy = dstY - srcY
print(f'{dx=}, {dy=}')

stepX = 1 if dx > 0 else -1
stepY = 1 if dy > 0 else -1
print(f'{stepX=}, {stepY=}')

x = srcX
x

In [ ]:
plt.figure()
plt.imshow(state, cmap='gray')

In [ ]:
srcX, srcY = 1, 1
dstX, dstY = 5, 3

srcX += 0.5
srcY += 0.5
dstX += 0.5
dstY += 0.5


dx = dstX - srcX
dy = dstY - srcY
print(f'{dx=}, {dy=}')

stepX = 1 if dx > 0 else -1
stepY = 1 if dy > 0 else -1
length = math.sqrt(dx*dx + dy*dy)

print(f'{stepX=}, {stepY=}, {length=}')

plt.figure(figsize=(8,8))
plt.imshow(state[:10,:10], cmap='gray')
plt.scatter(srcX-0.5, srcY-0.5, marker='X', color='black')
plt.scatter(dstX-0.5, dstY-0.5, marker='o', color='black')
plt.plot([srcX-0.5, dstX-0.5], [srcY-0.5, dstY-0.5])


x = math.ceil(srcX) if stepX > 0 else math.floor(srcX)

while 0 <= x < 20:
    y = srcY + (x - srcX) * dy / dx
    l = math.sqrt((x - srcX)**2 + (y - srcY)**2)
    print(f'{x=},{y=},{l=}')
    if l > length: break

    plt.scatter(x-0.5, y-0.5, marker='.', color='green')

    test_x = int(x + stepX*0.5)
    test_y = int(y)

    plt.plot([test_x-0.5,test_x+1-0.5], [test_y-0.5,test_y+1-0.5], color='green')
    plt.plot([test_x-0.5,test_x+1-0.5], [test_y+1-0.5,test_y-0.5], color='green')
    
    if state[test_y,test_x] == STATE_WALL: break
    
    x += stepX
    

y = math.ceil(srcY) if stepY > 0 else math.floor(srcY)

while 0 <= y < 20:
    x = srcX + (y - srcY) * dx / dy
    l = math.sqrt((x - srcX)**2 + (y - srcY)**2)
    print(f'{x=},{y=},{l=}')
    if l > length: break

    plt.scatter(x-0.5, y-0.5, marker='.', color='red')

    test_y = int(y + stepY*0.5)
    test_x = int(x)

    plt.plot([test_x-0.5,test_x+1-0.5], [test_y-0.5,test_y+1-0.5], color='red')
    plt.plot([test_x-0.5,test_x+1-0.5], [test_y+1-0.5,test_y-0.5], color='red')
    
    if state[test_y,test_x] == STATE_WALL: break

    y += stepY
    
plt.grid(color='gray', which='both', linestyle='-', linewidth=1)
plt.xticks(np.arange(0, 10, 1) + 0.5)
plt.yticks(np.arange(0, 10, 1) + 0.5)
None

In [ ]:
def is_visible(srcX, srcY, dstX, dstY):
    dx = dstX - srcX
    dy = dstY - srcY

    stepX = 1 if dx > 0 else -1
    stepY = 1 if dy > 0 else -1
    length = math.sqrt(dx*dx + dy*dy)

    plt.figure(figsize=(8,8))
    plt.imshow(state[:10,:10], cmap='gray')
    plt.scatter(srcX-0.5, srcY-0.5, marker='X', color='black')
    plt.scatter(dstX-0.5, dstY-0.5, marker='o', color='black')
    plt.plot([srcX-0.5, dstX-0.5], [srcY-0.5, dstY-0.5])
    plt.grid(color='gray', which='both', linestyle='-', linewidth=1)
    plt.xticks(np.arange(0, 10, 1) + 0.5)
    plt.yticks(np.arange(0, 10, 1) + 0.5)

    x = math.ceil(srcX) if stepX > 0 else math.floor(srcX)

    while 0 <= x < 20:
        y = srcY + (x - srcX) * dy / dx
        l = math.sqrt((x - srcX)**2 + (y - srcY)**2)
        if l >= length: break

        plt.scatter(x-0.5, y-0.5, marker='.', color='green')

        test_x = int(x + stepX*0.5)
        test_y = int(y)
        if abs(test_x - dstX) < 1e-6 and abs(test_y - dstY) < 1e-6: break

        plt.plot([test_x-0.5,test_x+1-0.5], [test_y-0.5,test_y+1-0.5], color='green')
        plt.plot([test_x-0.5,test_x+1-0.5], [test_y+1-0.5,test_y-0.5], color='green')
        
        if state[test_y,test_x] == STATE_WALL: return False
        
        x += stepX
        

    y = math.ceil(srcY) if stepY > 0 else math.floor(srcY)

    while 0 <= y < 20:
        x = srcX + (y - srcY) * dx / dy
        l = math.sqrt((x - srcX)**2 + (y - srcY)**2)
        if l >= length: break

        plt.scatter(x-0.5, y-0.5, marker='.', color='red')

        test_y = int(y + stepY*0.5)
        test_x = int(x)
        if abs(test_x - dstX) < 1e-6 and abs(test_y - dstY) < 1e-6: break
        
        plt.plot([test_x-0.5,test_x+1-0.5], [test_y-0.5,test_y+1-0.5], color='red')
        plt.plot([test_x-0.5,test_x+1-0.5], [test_y+1-0.5,test_y-0.5], color='red')
        
        if state[test_y,test_x] == STATE_WALL: return False

        y += stepY
            
    return True
    
srcX = 1
srcY = 1
dstX = 5
dstY = 3

visible = False
if dstX > srcX and is_visible(srcX + 0.5, srcY + 0.5, dstX, dstY + 0.5):
    visible = True
if dstX < srcX and is_visible(srcX + 0.5, srcY + 0.5, dstX + 1, dstY + 0.5):
    visible = True
if dstY > srcY and is_visible(srcX + 0.5, srcY + 0.5, dstX + 0.5, dstY):
    visible = True
if dstY < srcY and is_visible(srcX + 0.5, srcY + 0.5, dstX + 0.5, dstY + 1):
    visible = True
visible